In [1]:
import time
import math
import torch
import numpy as np
import quairkit as qkit
import matplotlib.pyplot as plt

from typing import List, Tuple
from tqdm import tqdm
from quairkit.database.state import zero_state, ghz_state
from quairkit.database.matrix import *
from quairkit.database import haar_orthogonal, haar_unitary, random_unitary_hermitian, pauli_group, random_clifford
from quairkit.core.state import to_state
from quairkit.circuit import Circuit
from quairkit.qinfo import permute_systems, partial_transpose, trace_distance, dagger, NKron, trace

qkit.set_dtype('complex128')
NUMPY_DTYPE = 'complex128'

In [2]:
def qrenn_cir(data_generators:torch.Tensor,
             trainable_qubits:int, 
             V_number_qubits:int, 
             layers:int, 
             is_conj:bool = False) -> Circuit:
    
    cir = Circuit(trainable_qubits + V_number_qubits)
    
    cir.h(qubits_idx=list(range(trainable_qubits, trainable_qubits+V_number_qubits)))
    for _ in range(layers):
        cir.universal_qudits(qubits_idx=list(range(trainable_qubits)))
        if is_conj:
            if _ % 2 == 0:
                cir.control_oracle(data_generators, system_idx = [list(range(trainable_qubits))] + 
                                    list(range(trainable_qubits, V_number_qubits +trainable_qubits)), gate_name='g(U)')
            else:
                cir.control_oracle(dagger(data_generators), system_idx = [list(range(trainable_qubits))] + 
                                    list(range(trainable_qubits, V_number_qubits +trainable_qubits)), gate_name='g(U^{\dagger})')
        else:
            cir.control_oracle(data_generators, system_idx = [list(range(trainable_qubits))] + 
                                    list(range(trainable_qubits, V_number_qubits +trainable_qubits)), gate_name='g(U)')
    # cir.u3(qubits_idx=list(range(trainable_qubits)))
    cir.universal_qudits(qubits_idx=list(range(trainable_qubits)))

    return cir


In [3]:
def mse_loss(v:qkit.State, num_labels:int, m:torch.Tensor):
    return torch.norm(trace(m @ v.density_matrix).squeeze() - num_labels, p=2) / len(num_labels)

def empirical_loss(v:qkit.State, num_labels:int, m:torch.Tensor):
    return torch.mean( - num_labels * torch.abs(trace(m @ v.density_matrix)))

def proj_loss(v:qkit.State, num_labels:int):
    P00 = (1.0+0.0j) * torch.tensor([[1,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
    P11 = (1.0+0.0j) * torch.tensor([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,1]])
    projplus = P00 + P11
    projminus = torch.eye(4) - projplus
    rho = v.trace(list(range(2,v.num_qubits)))
    rho = rho.density_matrix
    loss = 0.0
    for j in range(len(num_labels)):
        if num_labels[j] == -1:
            loss += torch.trace(rho[j] @ projminus)
        else:
            loss += torch.trace(rho[j] @ projplus)
    return -torch.real(loss) / len(num_labels)
    

# Training
def train_model_convergence(u:torch.Tensor,
                y_labels:torch.Tensor,
                trainable_qubits:int,
                input_state:qkit.State, 
                ITR:int = 100, 
                LR:float = 0.1, 
                slot:int = 10):
    
    assert u.shape[0] == y_labels.shape[0], "Number of Hams and y_labels should be the same"
    
    dim_h = u[0].shape[0]
    V_number_qubits = int(np.log2(dim_h))
    
    loss_list, time_list = [], []
    
    # initialize the model
    cir = qrenn_cir(u, trainable_qubits, V_number_qubits, layers=slot)
    
    # cir is a Circuit type
    opt = torch.optim.Adam(lr=LR, params=cir.parameters())

    # activate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, "min", factor=0.5)

    print("Training:")
    _ = 0
    while _ < ITR:

        start_time = time.time()
        opt.zero_grad()

        loss = proj_loss(cir(input_state), y_labels)

        loss.backward()  # compute gradients
        opt.step()  # update parameters
        scheduler.step(loss)  # activate scheduler

        loss = loss.item()
        loss_list.append(loss)
        time_list.append(time.time() - start_time)

        if scheduler.get_last_lr()[0]<2e-8:
            print(
                f"iter: {_}, loss: {loss:.8f}, lr: {scheduler.get_last_lr()[0]:.2E}, avg_time: {np.mean(time_list):.4f}s"
            )
            time_list = []
            break

        if _ % 100 == 0 or _ == ITR - 1:
            # print(
            #     f"iter: {_}, loss: {loss:.8f}, lr: {scheduler.get_last_lr()[0]:.2E}, avg_time: {np.mean(time_list):.4f}s"
            # )
            print(
                f"iter: {_}, loss: {loss:.8f}, lr: {scheduler.get_last_lr()[0]:.2E}, avg_time: {np.mean(time_list):.4f}s"
            )
            time_list = []
        _ += 1
    return cir, loss_list


def predict_model(u:torch.Tensor,
                  trainable_qubits:int, 
                  params:torch.Tensor,
                  input_state:qkit.State, 
                  slot:int,
                  scale:float=1.0):
    
    z0 = qkit.Hamiltonian([[scale, ",".join([f"Z{i}" for i in range(trainable_qubits)])]])
    
    dim_h = u[0].shape[0]
    V_number_qubits = int(np.log2(dim_h))

    # initialize the model
    cir = qrenn_cir(u, trainable_qubits, V_number_qubits, layers=slot)
    cir.update_param(params)

    return cir(input_state).expec_val(z0)

In [4]:
# Data generation
def uhermitian_data_generation(dim:int, num_samples:int) -> Tuple[torch.Tensor, torch.Tensor]:
    sour = np.random.randint(0, 2, num_samples)
    y_labels = np.where(sour == 0, -1, 1)
    data_U = np.zeros((num_samples, dim, dim), dtype=NUMPY_DTYPE)
    for y in range(num_samples):
        if sour[y]:
             # Randomly choose eigenvalues of +1 or -1
            eigenvalues = np.random.choice([1.0, -1.0], size=dim)
            D = np.diag(eigenvalues)
            
            # Generate a random unitary matrix U
            U = haar_unitary(dim)
            data_U[y, :, :] = U @ D @ U.conj().T

        else:
            data_U[y, :, :] = haar_unitary(dim)
    
    return torch.tensor(y_labels), torch.tensor(data_U)


def uhermitian_data_generation_erd(num_samples:int, V_number_qubits) -> Tuple[torch.Tensor, torch.Tensor]:
    n = 2**V_number_qubits
    sour = np.random.randint(0, 2, num_samples)
    y_labels = torch.tensor(np.where(sour == 0, -1, 1))
    data_U = torch.zeros((num_samples, n, n), dtype=torch.complex128)
    for y in range(num_samples):
        if sour[y]:
            eigenvalues = (torch.randint(0, 2, (n,)) * 2 -1).to(torch.complex128)
            D = torch.diag(eigenvalues)
            U = haar_unitary(n)
            
            # Generate a random unitary matrix U
            data_U[y, :, :] = U @ D @ U.conj().T
        else:
            
            mat = np.random.randn(n, n) + 1j * np.random.randn(n, n)
            for i in range(n):
                mat[i, i] = np.abs(mat[i, i])
                for j in range(i):
                    mat[i, j] = np.conj(mat[j, i])
                    
            data_U[y, :, :] = torch.matrix_exp(-1j * torch.tensor(mat, dtype=torch.complex128))
                
    return y_labels, data_U


def random_diag_data_generation(dim:int, num_samples:int) -> Tuple[torch.Tensor, torch.Tensor]:
    sour = np.random.randint(0, 2, num_samples)
    y_labels = np.where(sour == 0, -1, 1)
    data_U = np.zeros((num_samples, dim, dim), dtype=NUMPY_DTYPE)
    for y in range(num_samples):
        if sour[y]:
             # Randomly choose eigenvalues of +1 or -1
            eigenphase = 1*np.pi*np.random.random(size=dim)
            data_U[y, :, :] = np.diag(np.exp(1j*eigenphase))
        else:
            data_U[y, :, :] = haar_unitary(dim)
    
    return torch.tensor(y_labels), torch.tensor(data_U)


def random_pauli_data_generation(dim:int, num_samples:int) -> Tuple[torch.Tensor, torch.Tensor]:
    sour = np.random.randint(0, 2, num_samples)
    n = int(np.log2(dim))
    y_labels = np.where(sour == 0, -1, 1)
    data_U = np.zeros((num_samples, dim, dim), dtype=NUMPY_DTYPE)
    for y in range(num_samples):
        if sour[y]:
            data_U[y, :, :] = pauli_group(n)[np.random.randint(1, 4**n)]
        else:
            data_U[y, :, :] = haar_unitary(dim)
    
    return torch.tensor(y_labels), torch.tensor(data_U)

In [5]:
trainable_qubits = 2
V_number_qubits = 6

# data generation
train_samples = 100
test_samples = 500

# labels, data_raw = random_pauli_data_generation(2**V_number_qubits, train_samples+test_samples)
labels, data_raw = uhermitian_data_generation(2**V_number_qubits, train_samples+test_samples)
# labels, data_raw = uhermitian_data_generation_erd(train_samples+test_samples, V_number_qubits)
# labels, data_raw = random_diag_data_generation(2**V_number_qubits, train_samples+test_samples)

In [118]:
slot = 8
LR = 0.1
ITR = 1400
noise_rate = 0.05
input_state = zero_state(trainable_qubits+V_number_qubits)

# Create a shuffled copy of psi states
shuffled_indices = np.random.permutation(labels.shape[0])
data_raw = data_raw[shuffled_indices]
labels = labels[shuffled_indices]

y_train, x_train = labels[:train_samples], data_raw[:train_samples]
y_test, x_test = labels[train_samples:], data_raw[train_samples:]

for j in range(train_samples):
        if np.random.random() < noise_rate:
            y_train[j] = -y_train[j]
        else:
            pass

cir, loss_ = train_model_convergence(u=x_train,
                              y_labels=y_train,
                              trainable_qubits=trainable_qubits, 
                              input_state=input_state,
                              ITR = ITR, 
                              LR = LR, 
                              slot = slot)

Training:
iter: 0, loss: -0.56192090, lr: 1.00E-01, avg_time: 0.1652s
iter: 100, loss: -0.73371173, lr: 1.00E-01, avg_time: 0.1708s
iter: 200, loss: -0.73627283, lr: 1.00E-01, avg_time: 0.1714s
iter: 300, loss: -0.73671915, lr: 1.00E-01, avg_time: 0.1727s
iter: 400, loss: -0.73698366, lr: 1.00E-01, avg_time: 0.1777s
iter: 500, loss: -0.72947962, lr: 1.00E-01, avg_time: 0.1781s
iter: 600, loss: -0.73701254, lr: 1.00E-01, avg_time: 0.1751s
iter: 700, loss: -0.73696873, lr: 5.00E-02, avg_time: 0.1782s
iter: 800, loss: -0.73701765, lr: 5.00E-02, avg_time: 0.1750s
iter: 900, loss: -0.73701861, lr: 5.00E-02, avg_time: 0.1737s
iter: 1000, loss: -0.73701921, lr: 5.00E-02, avg_time: 0.1754s
iter: 1100, loss: -0.73701960, lr: 5.00E-02, avg_time: 0.1742s
iter: 1200, loss: -0.73701987, lr: 5.00E-02, avg_time: 0.1774s
iter: 1300, loss: -0.73702006, lr: 5.00E-02, avg_time: 0.1757s
iter: 1399, loss: -0.73702019, lr: 5.00E-02, avg_time: 0.1747s


In [119]:
# cir.plot()

In [120]:
h_train = NKron(*[z() for _ in range(trainable_qubits)])
h_data = NKron(*[torch.eye(2, dtype=torch.complex128) for _ in range(V_number_qubits)])
m = NKron(h_train, h_data)

v = cir(input_state)
diff = np.where(trace(m @ v.density_matrix).detach().squeeze().numpy().real >= 0.0, 1, -1) - y_train.numpy()
print(diff)
print(np.sum(diff == 0) / diff.shape[0])

[ 0  0  0  0  2  2  0  2  0  0  0  0  0 -2  0  2  0  0  0  2  0  0  2  0
  0 -2  2  0  0  0 -2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  2  0  0  0  0  0  0  0 -2  0 -2  0  0  0  0  0 -2  0  0  0  0  0
  0  0  2 -2  0  0  0  0  0  0  0  0  0 -2 -2  0  2  0 -2  0  0  0  0  0
 -2  0  0  0]
0.79


#### Testing

In [121]:
y_predict = predict_model(x_test,
                            trainable_qubits=trainable_qubits, 
                            params=cir.param, 
                            input_state=input_state,
                            slot=slot)
diff = np.where(y_predict.detach().numpy() >= 0, 1, -1) - y_test.numpy()
print(diff)
print(np.sum(diff == 0) / diff.shape[0])

[ 0  0  2  0  0  0  0  2  0 -2  0  0 -2  0  0  0  0  0 -2  0  2  0  0  0
  0  0  0  2  0  0  0  0  0  0  2  0  0  0  0  2  0  0  0  2  0  0  0  0
  0  0 -2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0
  0  2  0  0  0  0  0  0  0  2  0  0 -2  0  0  0  0  0  0  0  0  0  0 -2
  0  2  0  2  0  0 -2  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0  0 -2  0  0  2  0  0  0
  2  0  0  0 -2  0  0 -2  0  0  0  0  0  0 -2  0  0  0  0  0  0 -2  0 -2
  0  0 -2  0  0  0  0  0  0  0  0  0  2  0  0  0  0 -2  0  2  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  2  0  0  0  0
  0  0  0  0  0  0  0  0 -2  2  0  0  0  0  0  0  0 -2  0  0  2  0  0  0
  0  0 -2  0 -2  0  0  0 -2  0  0  2  0  0  0 -2  0  0 -2  0  2  0  0 -2
  2  0  0  0  0  0  2  0  2  0  0  0  0  0  0  0  0  2  0  0 -2  0 -2  0
  2  0  0  2  0  0  0  0  0  0 -2  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 -2  0  0  0  0  0  0 -2 -2  0